<h1 style="font-size:2.8rem;color:#1351b4;">Modelo de Classificação de Propostas</h1>

O modelo de MVP observado junto ao cliente é um sistema classificador/categorizador das propostas publicadas na plataforma do [Brasil Participativo](https://brasilparticipativo.presidencia.gov.br/processes/programas/f/2/), sendo as categorias baseadas Ministérios do Brasil. Mais detalhes sobre os componentes do sistema abaixo.

Ademais, foi cogitado a possibilidade de adicionar componentes complementares ao pipeline do sistema, como um classificador de discurso de ódio e um classificador em subcategorias (baseadas nas próprias temáticas das propostas). Esses componentes ainda estão no campo das ideias.

<a id='1'></a>

<p style="padding:10px;background:#1351b4;margin:10;color:white;font-size:3rem;text-align:center;border-radius: 1px 1px;overflow:hidden;font-weight:100">Sumário</p>

- [Extração dos dados](#data-extraction)
- [Pré-processamento textual](#pre-processing)
- [Treinamento do modelo](#training-model)

Consiste na apresentação de relatório sobre o desenvolvimento do projeto para o cliente.

Conforme a finalização dos procedimentos no repositório do Github, as alterações serão repassadas no dia da reunião, junto a equipe. Ele só entrará no documento oficial, assim que no término do modelo MVP.
 
Como esse relatório se trata de um projeto MVP (Produto com mínima viabilidade), ou seja, primeira versão de modelo, as novas ferramentas especificadas no repositório serão adicionadas com a finalização do MVP.

<a id='data-extraction'></a>

<p style="padding:10px;background:#1351b4;margin:10;color:white;font-size:3rem;text-align:center;border-radius: 1px 1px;overflow:hidden;font-weight:100">Extração dos Dados</p>

A primeira etapa para o desenvolvimento do projeto é a extração dos dados.
Essa etapa é a base para garantir que os dados obsoletos, vazios ou repetidos não sejam repassados para próxima etapa (pré-processamento). As propostas foram finalizadas no dia 15 de julho de 2023, conforme foi esclarecido nas reuniões, assim gerando uma base de dados de 8254 propostas.
Esses dados foram disponibilizados no link do rodapé do site, gerando uma pasta zip contendo arquivos em formato .csv.
Partindo da pasta zip baixada, foram realizadas algumas etapas que serão detalhadas a seguir:


<h1 style="font-size:1rem;">IMPORTAÇÃO DAS BIBLIOTECAS</h1>

In [ ]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
import zipfile

Essas bibliotecas que serão utilizadas no algoritmo, dentre delas, o uso da biblioteca Pandas responsável por fazer a criação da tabela e categorização e remoção das colunas e linhas vazias.

A biblioteca BeatifulSoup responsável por extrair arquivos HTML ou XML parser diretamente de sites e também usado aqui para remover tags HTML da coluna na tabela.
As outra bibliotecas são usadas como recursos que serão esclarecidos no decorrer das etapas.
 

<h1 style="font-size:1rem;">EXTRAÇÃO DOS DADOS DO ARQUIVO ZIP DO SITE</h1>

In [ ]:
# Extrai o arquivo zip do brasil participativo
with zipfile.ZipFile('/home/leandro/Documentos/Trabalho/model/brasilparticipativo.presidencia.gov.br-open-data.zip', 'r') as zip_ref:
    for arquivo in zip_ref.namelist():
        if arquivo.endswith('brasilparticipativo.presidencia.gov.br-open-data-proposals.csv'):
            zip_ref.extract(arquivo, 'model')
            break

Os arquivos presentes dentro da pasta zip são fornecidos para o algoritmo, através do ZipFile.

<h1 style="font-size:1rem;">LEITURA DAS PROPOSTAS</h1>

In [ ]:
# Le as propostas
propostas = pd.read_csv("model/brasilparticipativo.presidencia.gov.br-open-data-proposals.csv", delimiter=";")

O arquivo em formato .csv extraído da pasta zip anteriormente precisa ser lido e carregado em uma variável. Para leitura do aquivo é necessário o uso de leitor csv como apresentado no algoritmo.


<h1 style="font-size:1rem;">A LIMPEZA DE LINHAS VAZIAS NA TABELA</h1>

In [ ]:
#seleciona as colunas que precisa
propostas = pd.DataFrame(propostas, columns=['category/name/pt-BR','title/pt-BR','body/pt-BR'])

#retira as linhas que estao vazias
proposta = propostas.dropna()

Os dados contidos na variável “propostas”, são transformados em tabela, com ajuda da biblioteca Pandas. Para simplificação das informações, a tabela foi categorizada em 4 colunas com a limpeza de títulos duplicados e colunas em branco, além disso as linhas vazias foram retiradas nessa etapa.
  

<h1 style="font-size:1rem;">REMOÇÃO DAS TAGS HTML DA COLUNA E REINSERÇÃO DO TEXTO EXTRAÍDO</h1>

In [ ]:
# Coluna somente com as propostas
coluna_html = propostas['body/pt-BR']

# retira as tags html da coluna de propostas do csv
texto_extraido = []
for elemento_html in coluna_html:
    soup = BeautifulSoup(elemento_html, 'html.parser')
    texto = soup.get_text()
    texto_extraido.append(texto)

 # Sobrescreve a coluna com o texto extraído
propostas['body/pt-BR'] = texto_extraido  

A remoção das tags html  da coluna “propostas” são essenciais para o modelo de aprendizado de máquina. Com uso de um recurso da biblioteca BeautifulSoup, os títulos apagados por conter tags em html podem ser reescritos com os textos extraídos.

<h1 style="font-size:1rem;">CRIAÇÃO DO ARQUIVO COM AS ALTERAÇÕES E REMOÇÃO DO ANTERIOR</h1>

In [ ]:
#Retorna o novo csv limpo
propostas.to_csv('model/propostas.csv', index=False)

#Remove o csv antigo 
os.remove('/home/leandro/Documentos/Trabalho/model/brasilparticipativo.presidencia.gov.br-open-data-proposals.csv')

Após todas as etapas o ultimo procedimento é criar um novo arquivo .CSV com as alterações feitas e remover as alterações antigas.

O novo arquivo csv criado nesta etapa parte para etapa de pré-processamento dos dados. 

<a id='pre-processing'></a>

<p style="padding:10px;background:#1351b4;margin:10;color:white;font-size:3rem;text-align:center;border-radius: 1px 1px;overflow:hidden;font-weight:100">Pré-processamento Textual</p>

Anteriormente, os dados extraídos do link criaram um documento que simplificou a tabela com as propostas, retirando colunas e linhas vazias. Agora, o objetivo do pré-processamento é detalhar as informações importantes para o algoritmo responsável pelo aprendizado de máquina. 

Para se obter a categorização do texto é necessário aplicar regras ou retirar dados não importantes do texto, isso identifica as palavras – chave repassadas para a performasse de cálculo do modelo. Os textos presentes nas propostas passarão por uma técnica de tokenização(rotulagem) , que significa, que cada palavra presente será direcionada para dentro de um arranjo, ao estarem dentro dela, é possível identificar suas sintaxes e classes gramaticais, através da biblioteca Spacy. 

O Spacy permite que separar palavras iguais pelo aspecto semântico aplicado no contexto das frases, por exemplo, a palavra “laranja”, pode ter dois significados como uma fruta ou cor, sabendo disso podemos considerá-la um substantivo ou adjetivo, isso é importante para destacar apenas palavras de maior valor que será aplicado para categorização das propostas. Além disso, o Spacy possui um módulo de idioma, que no site é definido como “pipeline”, ela apresenta as características da gramática e sintaxe da língua correspondente, no projeto foi utilizado o idioma em português, O Spacy fornece um campo vasto de recursos para identificar as palavras na frase ou discurso do texto. O fato de poder identificar as palavras pelas suas classes gramaticais é uma parte crucial para limpar dados de texto sem valor ou ausentes para a próxima fase, ela evita erros desnecessários no algoritmo de aprendizado de máquina.

Há outra técnicas apresentadas no pré-processamento, como a lematização, que nada mais é organizar as palavras de semânticas semelhantes em um grupo, por exemplo, as palavras como (dar, dado, deu), aparecem no texto com padrões diferentes e cada um teria uma contagem individual, com a lematização as palavras seriam agrupadas em um conjunto defino “dar”, seria o lema das três  palavras. 

Além da tokenização (rotulagem) explicada anteriormente, as palavras que repetem em abundância e não possuem um teor valorizado para aplicação na próxima etapa como artigos, objetos diretos e indiretos, assim as palavras a, as, os , um, uns  e entre muitas outras são definidas como “Stop Words”, elas dificultariam o processo de cálculo no aprendizado de máquina devido a grande quantidade. Essas palavras serão excluídas no decorrer da fase de pré-processamento.

Essa etapa objetiva em uma otimização ou limpeza de dados de texto que não possuam valor para o processo de aprendizado de máquina. Essa limpeza, retira a “sujeira”, como valores em branco ou nulos que dificultariam ou reduziriam a precisão dos resultados do modelo, causando erros na próxima etapa do projeto.

No caso, podemos trabalhar com o dados estruturados, que são dados organizados em linhas e colunas, ou apenas os dados de texto que não possuem uma estrutura e é considerada como um saco de palavras, são textos sem pontuações, parágrafos e formatados em letras minusculas. No uso dados de texto é necessário o uso de técnicas para limpeza dos dados, como a remoção de tags URL e HTML, remoção de números, pronomes, siglas entre outros. Todas essas técnicas entram no escopo de pré-processamento de dados, devido a grande quantidade de dados precisam de atenção especial e detalhada para remover características menos importantes e ajudar na precisão do cálculo para o algoritmo de aprendizado de máquina.

<a id='training-model'></a>

<p style="padding:10px;background:#1351b4;margin:10;color:white;font-size:3rem;text-align:center;border-radius: 1px 1px;overflow:hidden;font-weight:100">Treinamento do Modelo</p>